In [ ]:
%matplotlib tk

In [ ]:
import pyxem as pxm
import numpy as np

# Load a dataset

In [ ]:
dp = pxm.load('/home/hremadmin/Downloads/nanowire_precession.hdf5')
dp = pxm.ElectronDiffraction(dp)

In [ ]:
scale = 0.035

In [ ]:
mask = dp.get_direct_beam_mask(8)

In [ ]:
dp = dp*np.invert(mask)

In [ ]:
dp.plot()

In [ ]:
dp = pxm.ElectronDiffraction(dp)

In [ ]:
dp.set_calibration(scale)

# Identify peaks

In [ ]:
peaks = dp.find_peaks_interactive()

In [ ]:
peaks = dp.find_peaks(method='max',
                      alpha=3.75,
                      size=10)

In [ ]:
peaks

### Store the unique peaks only

In [ ]:
pks = peaks.get_unique_vectors(distance_threshold=6*scale)

In [ ]:
np.shape(pks)

# Create VDF images of all unique peaks

In [ ]:
vdfs = peaks.get_vdf_images(electron_diffraction=dp, 
                            radius=scale*2,
                            unique_vectors=pks)

In [ ]:
vdfs.plot()

# New features

#0. Separate out all individual grains

#1. Calculte cross-correlation at zero displacement between all pairs of images




#2. Identify pairs with cross-corellations above some threshold to consider as from the same grain



#3. Identfiy gvectors corresponding to the vdfs from each grain




#4. Construct effective separate diffraction pattern for each grain



#5. Attempt indexation of reconstructed diffraction pattern

# Separate grains in the VDF images

### TODO: Enable plot of separate at i
Choose parameters and test separation on one VDF image at point i. Update parameters and check again at new point, if necessary.

In [ ]:
min_distance = 2
min_size = 7
max_size = vdfs.axes_manager[1].size*vdfs.axes_manager[2].size
threshold = 0.45
max_number_of_grains = 100
exclude_border = 0

Separate grains in the whole stack

In [ ]:
seps_stack = vdfs.separate_stack(min_distance=min_distance,
                                 threshold=threshold,
                                 min_size=min_size,
                                 max_size = max_size, 
                                 max_number_of_grains =max_number_of_grains,
                                 exclude_border = exclude_border)

In [ ]:
seps_stack

In [ ]:
stack=seps_stack.get_VDFgvectorStack(pks)

Made a new class to hold both images and vectors. To acess the stack: 
- stack.images gives all VDFs
- stack.vectors gives all the corresponding gvectors

In [ ]:
np.shape(stack.images)

Plot all the separated grains

In [ ]:
stack.images.plot()

# Image correlate all separated grains to merge VDFs and gvectors of the same grain

In [ ]:
stack_corr=stack.image_correlate_stack(corr_threshold=0.4)

Extract and plot all the vectors in stack_corr

In [ ]:
stack_corr.vectors.plot()

Plot vectors for the first VDF only: 

In [ ]:
stack_corr.vectors.plot(stack_corr.vectors.data[0])

In [ ]:
stack_corr.vectors.plot(stack_corr.vectors.data[1])

Create a signal from the gvectors

In [ ]:
Diff = stack_corr.get_virtual_electron_diffraction_signal(sm,distance_threshold=scale*2)

plot the images and gvectors with a shared navigation space.

In [ ]:
hs.plot.plot_signals([stack_corr.images,Diff])

### Manage the image correlatated merge stack: 
At this point, gvectors that are shared between two neighbouring grains is separated out as beloning to a new grain covering both of the adjacent grains. The virtual DPs of these adjacent grains do not contain the gvectors found in the overlapping grain... This is logically correct, since it might as well be an overlapping grain, but will give strange virtual DPS... 

Therefore, I had to add the possibility to sum or delete images and gvectors 'manually'. 

Another possibility is to look for grains with less than a certain amount of gvectors, e.g. 3, add those to overlapping grains, and delete this individual image. This can be done with thresholding, further down

stack_man = stack_corr.manage_images_and_gvectors_at_indices(image_add_indices = [[6,7],[12,13,14]],
                     gvectors_add_indices = [[1,0,2]],
                     image_delete_indices = [0,2,8,13,14])

Check that the number of images has been reduced correctly. 

### Threshold the merge_stack_corr

Threshold based on how many gvectors are found in the DP of the VDF image. Only keep those with sufficient number of gvectors. 

In [ ]:
stack_thresh=stack_corr.threshold_VDFgvectorStack(image_threshold = None,
                                                   gvector_threshold = 10)

Check that thenumer of images in stack has been reduced correctly

In [ ]:
np.shape(stack_thresh.images)

In [ ]:
stack_thresh.images.plot()

In [ ]:
sm.plot()

# Finally create virtual electron diffraction signal based on gvectors

In [ ]:
data = stack_thresh.get_virtual_electron_diffraction_signal(sm,2*scale)

In [ ]:
data.plot()

In [ ]:
np.shape(data)

In [ ]:
test_data = pc.ElectronDiffraction(data.data.reshape(2,2,144,144))

In [ ]:
np.shape(test_data)

In [ ]:
test_data.set_calibration(scale)

In [ ]:
test_data.plot()